# The Movies Dataset TF-IDF Content-Based Recommendation

##Понимание данных:

### Набор данных

Дан набор данных, содержащий информацию о 45,000 фильмах, выпущенных до июля 2017 года. Набор данных представлен на ресурсе Kaggle по ссылке https://www.kaggle.com/rounakbanik/the-movies-dataset
где представлено следующее описание составляющих файлов (выполнен перевод на русский язык):

`movies_metadata.csv:` Основной файл метаданных фильмов. Содержит информацию о 45 000 фильмов, представленных в наборе данных Full MovieLens. В таблицы представлены плакаты, фоны, бюджет, доход, даты выпуска, языки, страны-производители и компании.

`keywords.csv:` Содержит ключевые слова сюжета для наших фильмов MovieLens. Доступен в виде строкового объекта JSON.

`credits.csv:` Состоит из информации об актерах и съемках всех наших фильмов. Доступен в виде строкового объекта JSON. 

`links.csv:` Файл содержит идентификаторы TMDB и IMDB всех фильмов, представленных в наборе данных Full MovieLens.

`links_small.csv:` Содержит идентификаторы TMDB и IMDB небольшого подмножества из 9000 фильмов полного набора данных.

`ratings_small.csv:` Подмножество 100 000 оценок от 700 пользователей на 9 000 фильмов.

`ratings.csv `- файл, содержащий полный список оценок, выставленных пользователями фильмам



Рассмотрим подробнее две таблицы:

*   `movies_metadata.csv `
*   `ratings.csv `

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
! pip install --upgrade --no-cache-dir gdown

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14774 sha256=933e81e30d4f3296134fad477af62336cdba396dce344f5389dfd203a3b9cee1
  Stored in directory: /tmp/pip-ephem-wheel-cache-m75zchaf/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.2.2
    Uninstalling gdown-4.2.2:
      Successfully uninstalled gdown-4.2.2


In [ ]:
!gdown 18PC17_iR9f4WuGIKLSHU3drPt1jdCofA

Downloading...
From: https://drive.google.com/uc?id=18PC17_iR9f4WuGIKLSHU3drPt1jdCofA
To: /content/ratings.csv
100% 710M/710M [00:03<00:00, 199MB/s]


In [ ]:
!gdown 1wTPUbZx4C2zhdqzPk3yxKyVZJIxQcsvy

Downloading...
From: https://drive.google.com/uc?id=1wTPUbZx4C2zhdqzPk3yxKyVZJIxQcsvy
To: /content/movies_metadata.csv
100% 34.4M/34.4M [00:00<00:00, 168MB/s]


Начинаем с анализа датафрейма movies_metadata

In [ ]:
metadata = pd.read_csv("movies_metadata.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Задание 1

Удадите из датафрейма metadata строки, в которых отсутствует описание. Обратите внимание, что у некоторых фильмов формально описание есть, но там написано No overview found, No overview, No movie overview available, Released. Такие строки тоже нужно удалить. 

In [ ]:
metadata = pd.read_csv("movies_metadata.csv")
metadata.shape

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(45466, 24)

In [ ]:
metadata = metadata.dropna()
metadata.shape

(693, 24)

In [ ]:
my_list = [np.NaN, "No overview found.", "No overview", "No movie overview available", "Released", ""]

In [ ]:
metadata = metadata[~metadata['overview'].isin(my_list)]

In [ ]:
filter = (metadata['overview'].str.len() > 20)
metadata = metadata.loc[filter]

## Задание 2

Оставьте в датафрейме столбцы `'id', 'imdb_id', 'overview', 'title'`. Выведите 10 случайных строк.

In [ ]:
metadata = metadata[['id', 'imdb_id', 'overview', 'title']]
metadata.sample(10)

,id,imdb_id,overview,title
15910,43933,tt1470827,Six years ago NASA discovered the possibility ...,Monsters
35220,329833,tt1608290,Derek and Hansel are modelling again when an o...,Zoolander 2
16911,60307,tt1650043,"Back in middle school after summer vacation, G...",Diary of a Wimpy Kid: Rodrick Rules
1976,12230,tt0055254,When a litter of dalmatian puppies are abducte...,One Hundred and One Dalmatians
26502,58664,tt1052024,Based on the successful book series by R.L. St...,Mostly Ghostly
20815,76589,tt2027128,"An adaptation of Mark Waid's ""Tower of Babel"" ...",Justice League: Doom
18252,49026,tt1345836,Following the death of District Attorney Harve...,The Dark Knight Rises
7333,36670,tt0086006,James Bond returns as the secret agent 007 one...,Never Say Never Again
6723,11059,tt0317676,"Set on an island off the coast, a techno rave ...",House of the Dead
10386,8923,tt0385002,After being wrongfully expelled from Harvard U...,Green Street Hooligans


# Задание 3

In [ ]:
ratings = pd.read_csv("ratings.csv")

In [ ]:
ratings.head(10)

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556
5,1,1968,4.0,1425942148
6,1,2762,4.5,1425941300
7,1,2918,5.0,1425941593
8,1,2959,4.0,1425941601
9,1,4226,4.0,1425942228


In [ ]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26024289 entries, 0 to 26024288
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 794.2 MB


Убедимся в отсутствии пропусков

In [ ]:
pd.isnull(ratings).sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

Объедините датафреймы `metadata` и `ratings` в один. Обратите внимание, что `'id'` в  `metadata`, этот тот же самый идентификатор, что и `'movie_id'` и `ratings`. Объединять нужно по этому идентификатору (также обратите внимание на его тип данных).

In [ ]:
metadata['id'] = metadata['id'].astype(int) 

In [ ]:
temp = pd.merge(left=metadata, right=ratings, left_on='id', right_on='movieId', how='inner')

In [ ]:
temp.sample(10)

,id,imdb_id,overview,title,userId,movieId,rating,timestamp
566652,588,tt0384537,The eerie and deserted ghost town of Silent Hi...,Silent Hill,76948,588,3.0,843505846
558761,588,tt0384537,The eerie and deserted ghost town of Silent Hi...,Silent Hill,34729,588,4.0,1082611627
263273,708,tt0093428,James Bond helps a Russian General escape into...,The Living Daylights,151488,708,4.0,857221389
710677,2082,tt0373883,"After being committed for 17 years, Michael My...",Halloween,143847,2082,4.0,1031072545
128054,199,tt0117731,"The Borg, a relentless race of cyborgs, are on...",Star Trek: First Contact,35444,199,5.0,1077914022
558572,588,tt0384537,The eerie and deserted ghost town of Silent Hi...,Silent Hill,33658,588,4.0,842039497
46101,329,tt0107290,A wealthy entrepreneur secretly creates a them...,Jurassic Park,225878,329,5.0,949524744
555150,588,tt0384537,The eerie and deserted ghost town of Silent Hi...,Silent Hill,15221,588,4.0,835780048
275289,585,tt0198781,"James Sullivan and Mike Wazowski are monsters,...","Monsters, Inc.",43294,585,2.0,1111473840
718567,663,tt0890870,Jigsaw and his apprentice Amanda are dead. Now...,Saw IV,266044,663,4.0,879246774


## Задание 4

Появились ли пропуски в получившемся после объединения датафрейме? Если появились, то ответьте на вопрос "почему?" и удалите строки с пропусками.

Выведите размеры получившегося датафрейма и 10 случайных записей в нём.

In [ ]:
pd.isnull(temp).sum()

id           0
imdb_id      0
overview     0
title        0
userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [ ]:
temp.sample(10)

,imdb_id,overview,title,userId,movieId,rating,timestamp
515456,tt0372784,"Driven by tragedy, billionaire Bruce Wayne ded...",Batman Begins,192006,272,2.0,1337761554
220692,tt0070328,James Bond must investigate a mysterious murde...,Live and Let Die,100984,253,4.0,840230509
15338,tt0076759,Princess Leia is captured and held hostage by ...,Star Wars,189437,11,5.0,838053845
454705,tt0372183,When a CIA operation to purchase classified Ru...,The Bourne Supremacy,10393,2502,3.5,1092080791
26953,tt0107290,A wealthy entrepreneur secretly creates a them...,Jurassic Park,52461,329,4.0,842130683
764785,tt1053859,"Jake, the sole survivor of The Grudge 2 massac...",The Grudge 3,87147,1977,1.0,996030273
582094,tt0384537,The eerie and deserted ghost town of Silent Hi...,Silent Hill,160640,588,4.0,833397282
271859,tt0096928,"In the small town of San Dimas, a few miles aw...",Bill & Ted's Excellent Adventure,42419,1648,2.0,1473132804
351128,tt0120912,"Kay and Jay reunite to provide our best, last ...",Men in Black II,82708,608,5.0,942879353
639366,tt0479143,When he loses a highly publicized virtual boxi...,Rocky Balboa,17586,1246,4.5,1122963108


## Задание 5

Возьмите случайного пользователя (проверьте, чтобы у него было достаточное количество оценок). Сформируйте для этого пользователя рекомендацию методом коллаборативной фильтрации. Оцените качество это рекомендации.


Повторите эти рассчеты для большого числа пользователей и дайте интегральную оценку.

In [ ]:
from random import randrange

In [ ]:
temp.drop(columns='id', inplace=True)

KeyError: ignored

In [ ]:
movies_ratings = temp
movies_ids = movies_ratings.movieId.unique()
all_users = movies_ratings['userId'].value_counts()

In [ ]:
def getRandUserId():
  user_id = all_users.sample(1).index
  while len(movies_ratings[movies_ratings['userId'] == user_id[0]]) < 10:
    user_id = all_users.sample(1).index
  return user_id[0]

In [ ]:
getRandUserId()

234035

In [ ]:
user_id = getRandUserId()

In [ ]:
from scipy.spatial.distance import cdist
from math import ceil

In [ ]:
def getTempMoviesDf():
  temp = pd.DataFrame(index=movies_ids, columns=['rating'])
  temp['rating'] = 0.0
  return temp

In [ ]:
getTempMoviesDf()

,rating
710,0.0
755,0.0
5894,0.0
2292,0.0
11,0.0
...,...
64325,0.0
25961,0.0
48943,0.0
98439,0.0


In [ ]:
def getUserDf(uid):
  user_movies_df = movies_ratings.loc[movies_ratings['userId'] == uid]
  return user_movies_df

In [ ]:
getUserDf(user_id)

,imdb_id,overview,title,userId,movieId,rating,timestamp
22746,tt0107290,A wealthy entrepreneur secretly creates a them...,Jurassic Park,13573,329,4.5,1167775816
74644,tt0082340,"In 1997, the island of Manhattan has been wall...",Escape from New York,13573,1103,0.5,1167714259
81832,tt0108598,Gromit finds himself being pushed out of his r...,The Wrong Trousers,13573,531,3.0,1168206917
293325,tt0240772,"Less than 24 hours into his parole, charismati...",Ocean's Eleven,13573,161,5.0,1188084284
319357,tt0121765,"Ten years after the invasion of Naboo, the gal...",Star Wars: Episode II - Attack of the Clones,13573,1894,4.0,1167771018
326547,tt0258463,Wounded to the brink of death and suffering fr...,The Bourne Identity,13573,2501,3.5,1167715798
421441,tt0320691,Vampires and werewolves have waged a nocturnal...,Underworld,13573,277,3.0,1168207007
445754,tt0167190,"In the final days of World War II, the Nazis a...",Hellboy,13573,1487,0.5,1167774250
496031,tt0379593,"A comic, biting and revelatory documentary fol...",The Yes Men,13573,3396,3.0,1167713973
519106,tt0385700,Two years have passed since the final battle w...,Final Fantasy VII: Advent Children,13573,647,3.5,1167718299


In [ ]:
def getUserMoviesDf(uid):
  temp = getTempMoviesDf()
  user_df = getUserDf(uid)
  user_movies_rating_df = user_df[['movieId', 'rating']].set_index('movieId')
  temp.loc[user_movies_rating_df.index, ['rating']] = user_movies_rating_df.rating
  return temp

In [ ]:
getUserMoviesDf(user_id)

,rating
710,0.0
755,0.0
5894,0.0
2292,0.0
11,0.0
...,...
64325,0.0
25961,0.0
48943,0.0
98439,0.0


In [ ]:
getUserMoviesDf(user_id).value_counts()

rating
0.0       194
3.0         5
0.5         3
3.5         2
4.0         2
5.0         2
2.0         1
4.5         1
dtype: int64

In [ ]:
def getPreparedDf(uid, chunk_users):
  others = pd.crosstab(movies_ratings.movieId, movies_ratings.loc[movies_ratings.userId.isin(chunk_users), 'userId'])
  df = getUserMoviesDf(uid).drop(columns=['rating'])
  df.loc[others.index, others.columns] = others
  df[df.isna()] = 0.0
  return df

In [ ]:
other_uids = movies_ratings['userId'].unique().tolist()
other_uids = list(set(other_uids) - set([user_id]))
chunk = 3
chunk_size = 4000
chunk_users = other_uids[chunk * chunk_size: (chunk + 1) * chunk_size]
getPreparedDf(user_id, chunk_users)

,18891,18892,18893,18895,18897,18898,18899,18900,18903,18905,...,25246,25249,25250,25253,25254,25255,25257,25258,25260,25261
710,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
755,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5894,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2292,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64325,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48943,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98439,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def normalize(df):
  return df / 5

In [ ]:
def getDistance(df, user_movies_df):
  df = normalize(df)
  a = user_movies_df.values.reshape(1, -1)
  b = df.values.transpose()
  return cdist(XA=a, XB=b, metric='cosine')[0]

In [ ]:
other_uids = movies_ratings['userId'].unique().tolist()
other_uids = list(set(other_uids) - set([user_id]))
chunk = 3
chunk_size = 4000
chunk_users = other_uids[chunk * chunk_size: (chunk + 1) * chunk_size]
t = getPreparedDf(user_id, chunk_users)
u = getUserMoviesDf(user_id)
getDistance(t, u)

array([0.83338055, 1.        , 1.        , ..., 0.77180128, 0.82616909,
       1.        ])

In [ ]:
def countDistances(uid, chunks):
  other_uids = movies_ratings['userId'].unique().tolist()
  other_uids = list(set(other_uids) - set([uid]))
  chunk_size = ceil(len(other_uids) / chunks)
  target_df = getUserMoviesDf(uid)
  others_df = pd.DataFrame(columns=[other_uids])
  others_df.loc['dist'] = 0
  
  for chank in range(chunks):
    chank_users = other_uids[chank * chunk_size: (chank + 1) * chunk_size]
    t = getPreparedDf(uid, chank_users)
    d = getDistance(t, target_df)
    others_df.loc['dist', chank_users] = d
  return others_df

In [ ]:
distances = countDistances(user_id, 26)

In [ ]:
distances.loc['dist'].sort_values()[:10] 

29647     0.214508
260013    0.284678
225465    0.295904
133438    0.309925
53488     0.309925
15826     0.310016
117218    0.310016
218346    0.331431
115164    0.331955
169907    0.331955
Name: dist, dtype: object

In [ ]:
def getUserAvgRate(uid):
  target_df = getUserDf(user_id)
  return target_df['rating'].mean()

In [ ]:
getUserAvgRate(user_id)

3.0

In [ ]:
def getRecomendations(uid, dist, others_amount, rec_amount, exceptionId):
  target_df = getTempMoviesDf()
  ids_df = getUserDf(uid)['movieId'].values
  if(exceptionId):
    ids_df = ids_df[ids_df != exceptionId]
  target_df = target_df.loc[~target_df.index.isin(ids_df)]
  target_df['count'] = 0
  others = dist.loc['dist'].sort_values()[:others_amount]
  avg = getUserAvgRate(uid)
  for o in others.index:
    o_df = getUserDf(o)
    o_df = o_df.set_index('movieId')
    same_movies = list(set(target_df.index) & (set(o_df.index)))
    temp = pd.DataFrame(index = same_movies)  
    target_df.loc[temp.index, 'rating'] = target_df.loc[temp.index, 'rating'] + o_df.loc[temp.index, 'rating']
    target_df.loc[same_movies, 'count'] += 1
  #movie_top = target_df.sort_values('rating', ascending=False)[:rec_amount]
  movie_top_2 = target_df[target_df.rating / target_df['count'] > avg].sort_values('rating', ascending=False)[:rec_amount]
  #print(movie_top_2)
  #print(movie_top)
  return metadata.loc[(metadata.id.isin(movie_top_2.index))]

In [ ]:
getRecomendations(user_id, distances, 10, 3, None)

,id,imdb_id,overview,title
45,165,tt0096874,Marty and Doc are at it again in this wacky se...,Back to the Future Part II
144,2502,tt0372183,When a CIA operation to purchase classified Ru...,The Bourne Supremacy
201,1250,tt0259324,"In order to save his dying father, young stunt...",Ghost Rider


In [ ]:
getRecomendations(user_id, distances, 10, 3, 161)

,id,imdb_id,overview,title
45,165,tt0096874,Marty and Doc are at it again in this wacky se...,Back to the Future Part II
102,161,tt0240772,"Less than 24 hours into his parole, charismati...",Ocean's Eleven
144,2502,tt0372183,When a CIA operation to purchase classified Ru...,The Bourne Supremacy


###Оценка

In [ ]:
def getRandUsersIds(amount):
  users = []
  for i in range(amount):
    user = all_users.sample(1).index
    while len(temp[temp['userId'] == user[0]]) < 20 and user[0] not in users:
      user = all_users.sample(1).index
    users.append(user[0])
  return users

In [ ]:
hit_rate = np.zeros(15)
users = getRandUsersIds(15)

In [ ]:
users

[208241,
 118413,
 91373,
 199683,
 239882,
 224404,
 118147,
 61437,
 231910,
 173785,
 176951,
 32768,
 50585,
 126998,
 173641]

Для каждого из 15 пользователя отберем топ 5 фильмов и проверим будет ли хоть 1 из них в рекомендации из 10 фильмов, собранной на основе 10 ближайших пользователей.

In [ ]:
def countHitRate(users, hit_rate):
  for i in range(15):
    user = users[i]
    res = hit_rate[i]
    cur_succ = False
    excs = getUserDf(user).sort_values('rating', ascending=False).movieId[:5]
    dist = countDistances(user, 26)
    for e in excs:
      res = getRecomendations(user, distances, 10, 10, e).id.values
      if e in res:
        cur_succ = True
    if(cur_succ):
      hit_rate[i] = 1
    else:
      hit_rate[i] = 0
  return hit_rate

In [ ]:
hit_rate1 = countHitRate(users, hit_rate)

In [ ]:
hit_rate1.mean()

0.56

Для пользователей с 20 и более рекомендациями из 10 предложенных фильмов будет предложен минимум один из однозначно высоко оцененых фильмов в 10 рекомедациях в диапазоне ~ от 50% до 80% (0.5; 0.8)

## Задание 6

Используйте метод Term Frequency Inverse Document Frequency (TF-IDF), чтобы отфильтровать фильмы, похожие (используйте для этого косинусное расстояние) на те, которые пользователь высоко оценил. 

Оцените качество такой рекомендации.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
metadata = metadata.reset_index()
metadata = metadata.drop(columns = 'index')

In [ ]:
corpus = metadata['overview']
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tf_idf_data = tfidf_vectorizer.fit_transform(corpus)
cosine_sim = linear_kernel(tf_idf_data, tf_idf_data)

In [ ]:
series = pd.Series(metadata.index, index=metadata['title'])

In [ ]:
def intersection(lst1, lst2):
    return [value for value in lst1 if value in lst2]

In [ ]:
def getRecommendations2(title, cosine_sim, ids):
    i_title = series[title]
    sim_scores = list(enumerate(cosine_sim[i_title]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1].any(), reverse=True)
    sim_scores = sim_scores[1:11]
    movie_ids = [i[0] for i in sim_scores]
    return metadata['title'].iloc[movie_ids]

###Оценка

In [ ]:
hit_rate2 = np.zeros(50)
users2 = getRandUsersIds(50)

In [ ]:
def getHitRate2(hit_rate, users):
  for i in range(50):
    user = users[i]
    user_movies = temp[temp['userId'] == user]
    user_movies = user_movies[user_movies['rating'] > getUserAvgRate(user)]
    train = user_movies.sample(frac=0.7)
    test = user_movies[~user_movies['movieId'].isin(train['movieId'])]
    recommendations = []
    for title in train['title'].values:
      recommendations.append(getRecommendations2(title, cosine_sim, series)[:1].values[0]) 
    hit_rate[i] = len(intersection(recommendations, test['title'].values))
  return hit_rate

In [ ]:
hit_rate = getHitRate2(hit_rate2, users2)

In [ ]:
hit_rate.mean()

0.36

Для пользователей с 20 и более рекомендациями из 10 предложенных фильмов будут высоко оценены (> средней оценки пользователя) ~ от 2 до 4 шт. (0.2; 0.4)